In [1]:
#https://medium.com/neuralspace/kaggle-1-winning-approach-for-image-classification-challenge-9c1188157a86

In [2]:
import warnings
import seaborn as sns
import matplotlib.pylab as plt
import PIL
import pandas as pd
import os

import tensorflow as tf
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception, VGG16, ResNet50, InceptionV3
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception, preprocess_input


import numpy as np # linear algebra

from numpy.random import seed

Using TensorFlow backend.


In [3]:
warnings.filterwarnings('ignore')

SEED = 2
seed(SEED)
tf.set_random_seed(SEED)

In [4]:
print(os.listdir("../input"))

['test', 'class.csv', 'test.zip', 'train_crop_299', 'train.zip', 'sample_submission.csv', 'train', '.ipynb_checkpoints', 'train.csv', 'test.csv', 'test_crop_299', 'train_crop_224', 'test_crop_224']


In [5]:
DATA_PATH = '../input'
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')
TEST_IMG_PATH = os.path.join(DATA_PATH, 'test')

In [6]:
def crop_resize_boxing_img(img_name, margin=8, size=(299, 299)) :
    if img_name.split('_')[0] == "train" :
        PATH = TRAIN_IMG_PATH
        data = df_train
    elif img_name.split('_')[0] == "test" :
        PATH = TEST_IMG_PATH
        data = df_test
        
    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, \
                   ['bbox_x1','bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)

    width, height = img.size
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)

    return img.crop((x1,y1,x2,y2)).resize(size)

In [7]:
def crop_img(image_size, TRAIN_CROP_PATH, TEST_CROP_PATH):
    if not os.path.exists(TRAIN_CROP_PATH):
        os.mkdir(TRAIN_CROP_PATH)
        print('>>mkdir {}'.format(TRAIN_CROP_PATH))
        for i, row in df_train.iterrows():
            cropped = crop_resize_boxing_img(row['img_file'], size=(image_size, image_size))
            cropped.save(TRAIN_CROP_PATH+"/"+row['img_file'])
        print('>>train_crop completed')
    else:
        print('>>train_crop exist')
    if not os.path.exists(TEST_CROP_PATH):
        print('>>mkdir {}'.format(TEST_CROP_PATH))
        os.mkdir(TEST_CROP_PATH)
        for i, row in df_test.iterrows():
            cropped = crop_resize_boxing_img(row['img_file'], size=(image_size, image_size))
            cropped.save(TEST_CROP_PATH+"/"+row['img_file'])
        print('>>test_crop  completed')
    else:
        print('>>test_crop exist')

In [8]:
def split_traindf(df, train_size=0.6, stratify=True, label='class'):
    target = None
    if stratify:
        target = df[label].values
    X_train, X_val = train_test_split(df, train_size=train_size, random_state=SEED, stratify=target)
    X_train = X_train.reset_index(drop=True)
    X_val = X_val.reset_index(drop=True)
    return X_train, X_val

In [9]:
#https://github.com/yu4u/cutout-random-erasing
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        input_img = preprocess_input(input_img)
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [10]:
def get_generator(train_df, val_df, train_dir, valid_dir, test_df, test_dir, image_size, batch_size,valid_batch_size,
                 scale='rgb', target='class'):
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df, 
        directory=train_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
        shuffle=True,
        preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True)

    )
    validation_generator = valid_datagen.flow_from_dataframe(
        dataframe=val_df,
        directory=valid_dir,
        x_col = 'img_file',
        y_col = target,
        target_size=(image_size,image_size),
        batch_size=valid_batch_size,
        class_mode='categorical',
        seed=3,
        color_mode=scale,
        shuffle=True,
        processing_function=preprocess_input
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=test_dir,
        x_col='img_file',
        y_col=None,
        target_size= (image_size,image_size),
        color_mode=scale,
        class_mode=None,
        batch_size=batch_size,
        shuffle=False,
        preprocessing_function=preprocess_input
    )
    return train_generator, validation_generator, test_generator



In [11]:
def get_model(app, image_size, opt, num_class=196, lr=0.0001):
    if app=='Xception':
        application = Xception
    elif app=='VGG16':
        application = VGG16
    elif app=='ResNet50':
        application = ResNet50
    elif app=='InceptionV3':
        application = InceptionV3
    base_model = application(weights='imagenet', input_shape=(image_size,image_size,3), include_top=False)
    #base_model.trainable = False
    

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='sigmoid'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_class, activation='softmax'))
    #model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['acc'])

    return model

In [12]:
def get_model_path(model_dir, model_name):
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    from datetime import datetime
    now = datetime.now()
    date_time = now.strftime("%m%d_%H%M")
    model_path = model_dir + date_time + model_name + '.hdf5'
    print('>>model path to save: {}'.format(model_path))
    return model_path

In [13]:
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    

In [14]:
def plot_acc(history, model_name):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training Acc')
    plt.plot(epochs, val_acc, 'b', label='Validation Acc')
    plt.title('Training and validation accuracy' + model_name)
    plt.legend()
    plt.show()

    plt.plot(epochs, loss, 'bo', label='Traing loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Trainging and validation loss' + model_name)
    plt.legend()
    plt.show()


In [15]:

import keras
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps=0,
                             hold_base_rate_steps=0):
    """Cosine decay schedule with warm up period.

    Cosine annealing learning rate as described in:
      Loshchilov and Hutter, SGDR: Stochastic Gradient Descent with Warm Restarts.
      ICLR 2017. https://arxiv.org/abs/1608.03983
    In this schedule, the learning rate grows linearly from warmup_learning_rate
    to learning_rate_base for warmup_steps, then transitions to a cosine decay
    schedule.

    Arguments:
        global_step {int} -- global step.
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
    Returns:
      a float representing learning rate.

    Raises:
      ValueError: if warmup_learning_rate is larger than learning_rate_base,
        or if warmup_steps is larger than total_steps.
    """

    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                         'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + np.cos(
        np.pi *
        (global_step - warmup_steps - hold_base_rate_steps
         ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = np.where(global_step > warmup_steps + hold_base_rate_steps,
                                 learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                             'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * global_step + warmup_learning_rate
        learning_rate = np.where(global_step < warmup_steps, warmup_rate,
                                 learning_rate)
    return np.where(global_step > total_steps, 0.0, learning_rate)


class WarmUpCosineDecayScheduler(keras.callbacks.Callback):
    """Cosine decay with warmup learning rate scheduler
    """

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):
        """Constructor for cosine decay with warmup learning rate scheduler.

    Arguments:
        learning_rate_base {float} -- base learning rate.
        total_steps {int} -- total number of training steps.

    Keyword Arguments:
        global_step_init {int} -- initial global step, e.g. from previous checkpoint.
        warmup_learning_rate {float} -- initial learning rate for warm up. (default: {0.0})
        warmup_steps {int} -- number of warmup steps. (default: {0})
        hold_base_rate_steps {int} -- Optional number of steps to hold base learning rate
                                    before decaying. (default: {0})
        verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr))


In [16]:
def get_warmup_lr(base_lr, total_count, warmup_epoch=25):
    # Number of warmup epochs.
    warmup_epoch = 25

    total_steps = int(epoch * len(X_train) / batch_size)
    # Compute the number of warmup batches.
    warmup_steps = int(warmup_epoch * len(X_train) / batch_size)
    # Compute the number of warmup batches.
    warmup_batches = warmup_epoch * len(X_train) / batch_size

    # Create the Learning rate scheduler.
    warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=lr,
                                            total_steps=total_steps,
                                            warmup_learning_rate=0.0,
                                            warmup_steps=warmup_steps,
                                            hold_base_rate_steps=0)
    return warm_up_lr

In [17]:
def get_callback(patient, model_path, lr, total_count):
    callbacks = [
        EarlyStopping(monitor='val_loss',
                      patience=patient,
                      mode='min',
                      verbose=1),
        #ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = patient / 2, min_lr=0.00001, verbose=1, mode='min'),
        ModelCheckpoint(filepath=model_path,
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True,
                        mode='min'),
        get_warmup_lr(lr, total_count)
    ]
    return callbacks

In [18]:
def drop_119(df_train, keep=50, class_num='119'):
    class_119_idx = np.array(df_train[df_train['class']==class_num].index)
    np.random.shuffle(class_119_idx)
    return df_train.drop(class_119_idx[keep:])

In [19]:
K.clear_session()
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

In [20]:
model_type='Xception'
image_size = 299 if model_type=='Xception' or model_type=='InceptionV3' else 224

In [21]:
TRAIN_CROP_PATH = os.path.join(DATA_PATH, 'train_crop_' + str(image_size))
TEST_CROP_PATH = os.path.join(DATA_PATH, 'test_crop_' + str(image_size))
crop_img(image_size, TRAIN_CROP_PATH, TEST_CROP_PATH)
df_train["class"] = df_train["class"].astype('str')
df_train = df_train[['img_file', 'class']]
df_test = df_test[['img_file']]

>>train_crop exist
>>test_crop exist


In [22]:
print('>>befor drop : {}'.format(df_train.shape[0]))
df_train = drop_119(df_train)
print('>>after drop : {}'.format(df_train.shape[0]))

>>befor drop : 10016
>>after drop : 9982


In [23]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest',
    zca_whitening=True
)
valid_datagen = ImageDataGenerator(
    rescale=1. / 255
)
test_datagen = ImageDataGenerator(
    rescale=1. / 255
)

In [24]:
batch_size = 4
valid_batch_size=16

In [25]:
X_train, X_val = split_traindf(df_train.iloc[:, :], train_size=0.8, stratify=True)
nb_train_sample = X_train.shape[0]
nb_validation_sample = X_val.shape[0]
nb_test_sample = df_test.shape[0]
scale = 'rgb'
#scale = 'rgb'
train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                    val_df=X_val,
                                                    train_dir=TRAIN_CROP_PATH,
                                                    valid_dir=TRAIN_CROP_PATH,
                                                    test_df=df_test,
                                                    test_dir=TEST_CROP_PATH,
                                                    image_size=image_size,
                                                    batch_size=batch_size,
                                                    scale=scale,
                                                    valid_batch_size=valid_batch_size)

Found 7985 validated image filenames belonging to 196 classes.
Found 1997 validated image filenames belonging to 196 classes.
Found 6169 validated image filenames.


In [26]:
def weightsclaer(values, total_count):
    new_val = [total_count/x for x in values]
    min_val = min(new_val)
    new_val = [x/min_val for x in new_val]
    return new_val
labels_count = dict()
for label in X_train['class'].unique():
    labels_count[label] = sum(X_train['class']==label)
class_weight = None
#class_weight = weightsclaer(labels_count.values(), X_train.shape[0])

In [27]:
histories=[]
patient = 10
lr = 0.0001
epoch=300
model_dir = '../xception_model/'
model_name = '_3rd_Xception_cosinedecay'
model_path = get_model_path(model_dir, model_name)
model = get_model(app=model_type, image_size=image_size, opt=optimizers.RMSprop(lr=lr), lr=lr)
print('>>get model completed')

>>model path to save: ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
>>get model completed


In [28]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=get_steps(nb_train_sample, batch_size),
    epochs=epoch,
    validation_data=validation_gen,
    validation_steps=get_steps(nb_validation_sample, valid_batch_size),
    verbose=1,
    callbacks=get_callback(patient, model_path, lr, len(X_train)),
    class_weight=class_weight
)

Epoch 1/300
1997/1997 [==============================] - 262s 131ms/step - loss: 5.6393 - acc: 0.0051 - val_loss: 5.3774 - val_acc: 0.0050

Epoch 00001: val_loss improved from inf to 5.37742, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epoch 2/300
1997/1997 [==============================] - 257s 129ms/step - loss: 5.5265 - acc: 0.0063 - val_loss: 5.2446 - val_acc: 0.0125

Epoch 00002: val_loss improved from 5.37742 to 5.24461, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epoch 3/300
1997/1997 [==============================] - 258s 129ms/step - loss: 5.3945 - acc: 0.0055 - val_loss: 5.0814 - val_acc: 0.0351

Epoch 00003: val_loss improved from 5.24461 to 5.08142, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epoch 4/300
1997/1997 [==============================] - 258s 129ms/step - loss: 5.1970 - acc: 0.0131 - val_loss: 4.8026 - val_acc: 0.0876

Epoch 00004: val_loss improved from 5.08142 to 4.80256

KeyboardInterrupt: 

In [29]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255
)
valid_datagen = ImageDataGenerator(
    rescale=1. / 255
)
test_datagen = ImageDataGenerator(
    rescale=1. / 255
)

train_gen, validation_gen, test_gen = get_generator(train_df=X_train,
                                                    val_df=X_val,
                                                    train_dir=TRAIN_CROP_PATH,
                                                    valid_dir=TRAIN_CROP_PATH,
                                                    test_df=df_test,
                                                    test_dir=TEST_CROP_PATH,
                                                    image_size=image_size,
                                                    batch_size=batch_size,
                                                    scale=scale,
                                                    valid_batch_size=valid_batch_size)

Found 7985 validated image filenames belonging to 196 classes.
Found 1997 validated image filenames belonging to 196 classes.
Found 6169 validated image filenames.


In [30]:
lr = 0.0001
model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=lr), metrics=['acc'])
model.load_weights(model_path)#0.34662

In [31]:
print('start')

start


In [32]:
patient = 10
history_2nd = model.fit_generator(
    train_gen,
    steps_per_epoch=get_steps(nb_train_sample, batch_size),
    epochs=epoch,
    validation_data=validation_gen,
    validation_steps=get_steps(nb_validation_sample, valid_batch_size),
    verbose=1,
    callbacks=get_callback(patient, model_path, lr, len(X_train)),
    class_weight=class_weight
)

Epoch 1/300
1997/1997 [==============================] - 264s 132ms/step - loss: 0.1853 - acc: 0.9569 - val_loss: 0.4661 - val_acc: 0.8668

Epoch 00001: val_loss improved from inf to 0.46615, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epoch 2/300
1997/1997 [==============================] - 256s 128ms/step - loss: 0.1899 - acc: 0.9546 - val_loss: 0.4574 - val_acc: 0.8683

Epoch 00002: val_loss improved from 0.46615 to 0.45740, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epoch 3/300
1997/1997 [==============================] - 256s 128ms/step - loss: 0.1838 - acc: 0.9563 - val_loss: 0.4592 - val_acc: 0.8683

Epoch 00003: val_loss did not improve from 0.45740
Epoch 4/300
1997/1997 [==============================] - 258s 129ms/step - loss: 0.1794 - acc: 0.9557 - val_loss: 0.4454 - val_acc: 0.8693

Epoch 00004: val_loss improved from 0.45740 to 0.44542, saving model to ../xception_model/0625_0435_3rd_Xception_cosinedecay.hdf5
Epo

KeyboardInterrupt: 

In [ ]:
model.load_weights(model_path) #loss 0.29xx

In [ ]:
test_gen.reset()
prediction = model.predict_generator(
    generator=test_gen,
    steps = get_steps(nb_test_sample, batch_size),
    verbose=1
)
predicted_class_indices=np.argmax(prediction, axis=1)

In [ ]:
# Generator class dictionary mapping

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_labels = [labels[k] for k in predicted_class_indices]

submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions_labels
submission_fname = "submissions/submissions_{}.csv".format(model_name)
submission.to_csv(submission_fname, index=False)
submission.head()
print('[*]sumission saved at {}'.format(submission_fname))
#89318

base : batch 8 + more augmentation  + nodecay  + imgsize (224->299)

rmsprop ->

adam -> 

image generate 

tip1 : https://towardsdatascience.com/deep-learning-unbalanced-training-data-solve-it-like-this-6c528e9efea6

tip2 : https://imgaug.readthedocs.io/en/latest/source/installation.html#installation-in-pip


cosine learning rate decay

https://www.dlology.com/blog/bag-of-tricks-for-image-classification-with-convolutional-neural-networks-in-keras/

https://machinelearningmastery.com/snapshot-ensemble-deep-learning-neural-network/


progressive

https://towardsdatascience.com/boost-your-cnn-image-classifier-performance-with-progressive-resizing-in-keras-a7d96da06e20


check error visual

https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/


fine tuning-inception v3

https://gist.github.com/didacroyo/839bd1dbb67463df8ba8fb14eb3fde0c


image data generator

contrast_stretching=True, adaptive_equalization=True, histogram_equalization=True